In [ ]:
from flask import Flask, request, jsonify, make_response
from flask_sqlalchemy import SQLAlchemy
import uuid
from flask_bcrypt import Bcrypt
import jwt
import datetime
from functools import wraps

app = Flask(__name__)

app.config['SECRET_KEY'] = 'thisissecretkey'
app.config['SQLALCHEMY_DATABASE_URI'] = app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///members.db'
bcrypt = Bcrypt(app)
db = SQLAlchemy(app)

class Member(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    member_id = db.Column(db.String(50), unique=True)
    name = db.Column(db.String(50))
    password = db.Column(db.String(80))
    admin = db.Column(db.Boolean)
    
class Periodnote(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    mpain = db.Column(db.Boolean)
    colour = db.Column(db.String(50))
    note_body = db.Column(db.String(100))
    note_memberid = db.Column(db.Integer, db.ForeignKey('member.id'), nullable=False)
    member = db.Column(db.String(255))

def token_required(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        token = None

        if 'x-access-token' in request.headers:
            token = request.headers['x-access-token']

        if not token:
            return jsonify({'message' : 'Token is missing!'}), 401

        try: 
            data = jwt.decode(token, app.config['SECRET_KEY'])
            current_member = Member.query.filter_by(member_id=data['member_id']).first()
        except:
            return jsonify({'message' : 'Token is invalid!'}), 401

        return f(current_member, *args, **kwargs)

    return decorated   

@app.route('/api/member', methods=['POST'])
@token_required
def create_member(current_member):
#def create_member():
    if not current_member.admin:
        return jsonify({'message' : 'Cannot perform that function!'}) 
    data = request.get_json()
    hashed_password = bcrypt.generate_password_hash(data['password'])
    new_member = Member(member_id=str(uuid.uuid4()), name=data['name'], password=hashed_password, admin=False)
    db.session.add(new_member)
    db.session.commit()
    return jsonify({'message' : 'New member created!'})
    
@app.route('/api/member', methods=['GET'])
@token_required
def get_all_members(current_member):

    if not current_member.admin:
        return jsonify({'message' : 'Cannot perform that function!'})

    members = Member.query.all()

    output = []

    for member in members:
        member_data = {}
        member_data['member_id'] = str(member.member_id)
        member_data['name'] = member.name
        member_data['password'] = str(member.password)
        member_data['admin'] = member.admin
        output.append(member_data)

    return jsonify({'members' : output})


@app.route('/api/member/<member_id>', methods=['GET'])
@token_required
def get_one_member(current_member, member_id):

    if not current_member.admin:
        return jsonify({'message' : 'Cannot perform that function!'})

    member = Member.query.filter_by(member_id=member_id).first()

    if not member:
        return jsonify({'message' : 'No member found!'})

    member_data = {}
    member_data['member_id'] = str(member.member_id)
    member_data['name'] = member.name
    member_data['password'] = str(member.password)
    member_data['admin'] = member.admin

    return jsonify({'member' : member_data})

@app.route('/api/member/<member_id>', methods=['PUT'])
@token_required
def promote_member(current_member, member_id):
    
    if not current_member.admin:
        return jsonify({'message' : 'Cannot perform that function!'})

    member = Member.query.filter_by(member_id=member_id).first()

    if not member:
        return jsonify({'message' : 'No member found!'})

    member.admin = True
    db.session.commit()

    return jsonify({'message' : 'The member has been promoted!'})

@app.route('/api/member/<member_id>', methods=['DELETE'])
@token_required
def delete_member(current_member, member_id):
    
    if not current_member.admin:
        return jsonify({'message' : 'Cannot perform that function!'})

    member = Member.query.filter_by(member_id=member_id).first()

    if not member:
        return jsonify({'message' : 'No member found!'})

    db.session.delete(member)
    db.session.commit()

    return jsonify({'message' : 'The member has been deleted!'})

@app.route('/api/login')
def login():
    auth = request.authorization

    if not auth or not auth.username or not auth.password:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    member = Member.query.filter_by(name=auth.username).first()

    if not member:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    if bcrypt.check_password_hash(member.password, auth.password):
        token = jwt.encode({'member_id' : member.member_id, 'exp' : datetime.datetime.utcnow() + datetime.timedelta(minutes=30)}, app.config['SECRET_KEY'])

        return jsonify({'token' : token.decode('UTF-8')})

    return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

@app.route('/api/periodnote', methods=['GET'])
@token_required
def get_all_periodnote(current_member):
    #periodnotes = Periodnote.query.filter_by(note_memberid=current_member.id).all()
    periodnotes = Periodnote.query.all()
    
    output = []

    for periodnote in periodnotes:
        periodnote_data = {}
        periodnote_data['id'] = periodnote.id
        periodnote_data['title'] = periodnote.title
        periodnote_data['mpain'] = periodnote.mpain
        periodnote_data['colour'] = periodnote.colour
        periodnote_data['note_body'] = periodnote.note_body
        periodnote_data['note_memberid'] = periodnote.note_memberid
        periodnote_data['member'] = periodnote.member
        output.append(periodnote_data)

    return jsonify({'periodnotes' : output})

@app.route('/api/periodnote', methods=['POST'])
@token_required
def create_periodnote(current_member):
    data = request.get_json()

    new_periodnote = Periodnote(title=data['title'], mpain=False, colour=data['colour'], note_body=data['note_body'], note_memberid=current_member.id, member=current_member.name)
    db.session.add(new_periodnote)
    db.session.commit()

    return jsonify({'message' : "Periodnote created!"})

@app.route('/api/periodnote/<periodnote_id>', methods=['GET'])
#retrive the log-ined member's note 
@token_required
def get_one_periodnote(current_member, periodnote_id):
    periodnote = Periodnote.query.filter_by(id=periodnote_id, note_memberid=current_member.id).first()

    if not periodnote:
        return jsonify({'message' : 'No periodnote found!'})

    periodnote_data = {}
    periodnote_data = {}
    periodnote_data['id'] = periodnote.id
    periodnote_data['title'] = periodnote.title
    periodnote_data['mpain'] = periodnote.mpain
    periodnote_data['colour'] = periodnote.colour
    periodnote_data['note_body'] = periodnote.note_body
    periodnote_data['note_memberid'] = periodnote.note_memberid
    periodnote_data['member'] = periodnote.member

    return jsonify(periodnote_data)

@app.route('/api/periodnote/<periodnote_id>', methods=['PUT'])
@token_required
def mpain_periodnote(current_member, periodnote_id):
    periodnote = Periodnote.query.filter_by(id=periodnote_id, note_memberid=current_member.id).first()

    if not periodnote:
        return jsonify({'message' : 'No periodnote found!'})

    periodnote.mpain = True
    db.session.commit()

    return jsonify({'message' : 'Suffered from period pain this month!'})


@app.route('/api/periodnote/<periodnote_id>', methods=['DELETE'])
@token_required
def delete_periodnote(current_member, periodnote_id):
    periodnote = Periodnote.query.filter_by(id=periodnote_id, note_memberid=current_member.id).first()

    if not periodnote:
        return jsonify({'message' : 'No periodnote found!'})

    db.session.delete(periodnote)
    db.session.commit()

    return jsonify({'message' : 'Periodnote deleted!'})

if __name__ == '__main__':
    app.run('localhost', port=9021)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\Users\User\anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9021/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jan/2022 12:16:51] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [17/Jan/2022 12:16:51] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Jan/2022 12:16:59] "GET /api/member HTTP/1.1" 401 -
127.0.0.1 - - [17/Jan/2022 12:17:50] "POST /api/member HTTP/1.1" 401 -
127.0.0.1 - - [17/Jan/2022 12:17:58] "GET /api/login HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 12:18:05] "POST /api/member HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 12:18:15] "GET /api/login HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 12:18:22] "POST /api/member HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022 12:18:32] "GET /api/login HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2022